# Tensorflow 실습 1

python의 딥러닝 라이브러리인 Tensorflow에 대해 알아보자.

### 기초 문법과 예제

Tensorflow 는 graph 로 연산을 나타내는 프로그래밍 시스템
- Graph 의 node 는 연산, 즉 operation(op) 을 수행
- Graph 의 edge 는 tensor 의 흐름을 나타냄
- Autograph 및 eager excution을 통해, "define by run" 방식이 default로 제공
텐서 (Tensor)
- Numpy 의 ndarray 와 비슷한 다차원 배열로, tensorflow 에서의 데이터를 표현하는 구조
- 그래프 내 operation 에서 tensor 가 전달됨

케라스 (Keras)
- 기존에 존재하는 딥러닝 프레임워크로, 쉽게 이해할수 있는 코드로 구성되는 것이 특징
- Tensorflow에서 이를 적극적으로 활용하기로 결정하여, tf.keras라는 고수준 API로 다양한 layer를 사용할수 있도록 함



###Tensorflow의 간단한 연산 예시
- add, square, reduce_sum 등 간단한 함수들을 사용 가능

- 연산의 결과는 tf.Tensor로 나오며, input 값의 형태에 따라 자동으로 dtype이 배정됨

- 각각의 tf.Tensor는 shape와 dtype을 가지고 있음

In [ ]:
import tensorflow as tf

# constant tensor
print(tf.constant([3, 7]))

# 더하기
print(tf.add(1, 2))

# vector 더하기
print(tf.add([1, 2], [3, 4]))

# 제곱
print(tf.square(5.0))

# 합
print(tf.reduce_sum([1, 2, 3]))

# 각각 제곱 후 더하기 (연산자 오버로딩(overloading) 지원)
print(tf.square(2) + tf.square(3))

In [ ]:
# matrix 곱셈
a = tf.constant([[2],[3]])
b = tf.constant([[3,7]])
x = tf.matmul(a,b)
print(a.shape)
print(b.shape)
print(x)

# tensor shape 
print(x.shape)

# tensor data type
print(x.dtype)

###tensorflow와 numpy의 호환성
- list 또는 numpy array를 tf.Tensor로 변환할 때는, tf.convert_to_tensor 사용
- tensorflow 연산은 numpy array를 자동으로 tf.Tensor로 변환하여 사용
- 반대로, numpy 연산은 tf.Tensor를 numpy array로 변환함

In [ ]:
import numpy as np

ndarray = np.ones([3, 3])
# .convert_to_tensor 함수는 list, ndarray를 직접 텐서로 변환
print(tf.convert_to_tensor([[1,2],[3,4]], dtype=tf.float64))
print(tf.convert_to_tensor(ndarray, dtype=tf.int32))

# 텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환
tensor = tf.multiply(ndarray, 2)
print(tensor)

# 그리고 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환
print(np.add(tensor, 1))

# .numpy() 메서드는 텐서를 넘파이 배열로 변환
print(tensor.numpy())

### Variable (변수)
- 학습할 수 있는 parameter를 variable이라는 형태의 텐서로 표현함




In [ ]:
# tf.Vaiable()을 통해, input으로 list, numpy array, tensor 등을 넣어주면, variable로 변환
my_variable = tf.Variable(tf.zeros([2, 3]))
print(my_variable)

v = tf.Variable([[2,3],[3,4]])
print(v)

# variable을 수식에 사용하면, 자동적으로 tf.Tensor로 변환되어 값을 표현
w = v + 1
print(w)
print(v)

# variable에 .read_value()를 이용할 경우, 현재 변수 값을 명시적으로 읽어올 수 있음
print(v.read_value())


In [ ]:
# class에 tf.Module 등을 상속할 경우 .variables() 함수로 class가 보유한 변수 목록을 불러올 수 있음
class MyModuleOne(tf.Module):
    def __init__(self):
        self.v0 = tf.Variable(1.0)
        self.vs = [tf.Variable(x) for x in range(2)]

m = MyModuleOne()
print(m.variables)
print(len(m.variables))

### 자동 미분 (gradient tape의 사용법)
- gradient tape를 이용하면, variable에 대한 연산을 순서대로 모두 저장하고 자동으로 gradient를 계산하는 것이 가능
- with 구문 안의 연산을 tape에 저장하면, tape.gradient(target, sources)로 우리가 원하는 형태의 미분 값을 계산할 수 있음

In [ ]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  y = x**2

# dy/dy = 2x 
dy_dx = tape.gradient(y, x)
dy_dx.numpy()


- 위의 미분 값은 scalar 형태이지만, gradient는 tensor형태도 될수 있음
- sources 부분에 list를 넣으면 list가 출력되고, dictionary를 넣으면 dictionary 형태로 출력됨

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b  ## 참고: tf.matmul(x, w) 와 x @ w 동일함
  loss = tf.reduce_mean(y**2)

[dl_dw, dl_db] = tape.gradient(loss, [w, b])
print(w.shape)
print(dl_dw.shape)

In [ ]:
my_vars = {
    'w': w,
    'b': b
}
grad = tape.gradient(loss, my_vars)
print(grad)

###Device
- tensorflow로 설치할 경우 CPU만 사용 가능
- tensorflow-gpu로 설치할 경우 CPU, GPU 모두 사용 가능
- with tf.device("디바이스 종류 및 번호")
위의 코드를 통해, 연산이 실행되는 device를 지정할 수 있음
- 같은 연산일지라도, CPU보다는 GPU에서 훨씬 빠른 속도를 보임
- Colab에서 GPU 설정하기: 런타임 -> 런타임 유형 변경 -> 하드웨어 가속기 GPU로 세팅

In [ ]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(100):
    tf.matmul(x, x)

  result = time.time()-start
  print("100 loops: {:0.2f}ms".format(1000*result))

# CPU에서 강제 실행합니다.
print(tf.config.list_physical_devices('CPU'))
if tf.config.list_physical_devices('CPU'):
  print("On CPU:")
  with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# GPU #0가 이용가능시 GPU #0에서 강제 실행합니다.
print(tf.config.list_physical_devices('GPU'))
if tf.config.list_physical_devices('GPU'):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)